<a href="https://colab.research.google.com/github/chenchongsong/udacity-cs344-colab/blob/main/notebook/udacity_cs344_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Homework 3 for Udacity CS344 Course, Intro to Parallel Programming
# clone the code repo,
!git clone https://github.com/chenchongsong/udacity-cs344-colab
!pip install git+git://github.com/depctg/nvcc4jupyter.git

# load cuda plugin
%config NVCCPluginV2.static_dir = True
%config NVCCPluginV2.relative_dir = "udacity-cs344-colab/src/HW3"
%load_ext nvcc_plugin

# change to work directory, generate makefiles
!mkdir udacity-cs344-colab/build
%cd udacity-cs344-colab/build
!cmake ../src

In [ ]:
%%cuda --name student_func.cu

/* Udacity Homework 3
   HDR Tone-mapping

  Background HDR
  ==============

  A High Dynamic Range (HDR) image contains a wider variation of intensity
  and color than is allowed by the RGB format with 1 byte per channel that we
  have used in the previous assignment.

  To store this extra information we use single precision floating point for
  each channel.  This allows for an extremely wide range of intensity values.

  In the image for this assignment, the inside of church with light coming in
  through stained glass windows, the raw input floating point values for the
  channels range from 0 to 275.  But the mean is .41 and 98% of the values are
  less than 3!  This means that certain areas (the windows) are extremely bright
  compared to everywhere else.  If we linearly map this [0-275] range into the
  [0-255] range that we have been using then most values will be mapped to zero!
  The only thing we will be able to see are the very brightest areas - the
  windows - everything else will appear pitch black.

  The problem is that although we have cameras capable of recording the wide
  range of intensity that exists in the real world our monitors are not capable
  of displaying them.  Our eyes are also quite capable of observing a much wider
  range of intensities than our image formats / monitors are capable of
  displaying.

  Tone-mapping is a process that transforms the intensities in the image so that
  the brightest values aren't nearly so far away from the mean.  That way when
  we transform the values into [0-255] we can actually see the entire image.
  There are many ways to perform this process and it is as much an art as a
  science - there is no single "right" answer.  In this homework we will
  implement one possible technique.

  Background Chrominance-Luminance
  ================================

  The RGB space that we have been using to represent images can be thought of as
  one possible set of axes spanning a three dimensional space of color.  We
  sometimes choose other axes to represent this space because they make certain
  operations more convenient.

  Another possible way of representing a color image is to separate the color
  information (chromaticity) from the brightness information.  There are
  multiple different methods for doing this - a common one during the analog
  television days was known as Chrominance-Luminance or YUV.

  We choose to represent the image in this way so that we can remap only the
  intensity channel and then recombine the new intensity values with the color
  information to form the final image.

  Old TV signals used to be transmitted in this way so that black & white
  televisions could display the luminance channel while color televisions would
  display all three of the channels.


  Tone-mapping
  ============

  In this assignment we are going to transform the luminance channel (actually
  the log of the luminance, but this is unimportant for the parts of the
  algorithm that you will be implementing) by compressing its range to [0, 1].
  To do this we need the cumulative distribution of the luminance values.

  Example
  -------

  input : [2 4 3 3 1 7 4 5 7 0 9 4 3 2]
  min / max / range: 0 / 9 / 9

  histo with 3 bins: [4 7 3]

  cdf : [4 11 14]


  Your task is to calculate this cumulative distribution by following these
  steps.

*/

#include "utils.h"

const int BLOCK_SIZE = 1024;

__device__ float _min(float a, float b) {
  return a < b ? a : b;
}

__device__ float _max(float a, float b) {
  return a > b ? a : b;
}

__global__ void shmem_reduce(float * d_out, const float * d_in, int numElem, bool isMin) {
  // sdata is allocated in the kernel call: 3rd arg to <<<b, t, shmem>>>
  extern __shared__ float sdata[];

  int myId = threadIdx.x + blockDim.x * blockIdx.x;
  int tid  = threadIdx.x;

  // load shared mem from global mem
  if (myId >= numElem) {
    sdata[tid] = d_in[0];  // dummy init (does not modify the final result)
  } else {
    sdata[tid] = d_in[myId];
  }
  __syncthreads();            // make sure entire block is loaded!

  // do reduction in shared mem
  for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
    if (tid < s) {
      sdata[tid] = isMin ? _min(sdata[tid], sdata[tid + s]) : _max(sdata[tid], sdata[tid + s]);
    }
    __syncthreads();        // make sure all adds at one stage are done!
  }

  // only thread 0 writes result for this block back to global mem
  if (tid == 0) {
    d_out[blockIdx.x] = sdata[0];
  }
}

float reduce(const float* const d_logLuminance, int numElem, bool isMin) {
  // declare GPU memory pointers
  float* d_intermediate;
  float* d_out;
  // allocate GPU memory
  int numBlock = ceil(1.0f * numElem / BLOCK_SIZE);
  cudaMalloc(&d_intermediate, numBlock * sizeof(float));  // overallocated
  cudaMalloc(&d_out, sizeof(float));

  shmem_reduce<<<numBlock, BLOCK_SIZE, BLOCK_SIZE * sizeof(float)>>>(d_intermediate, d_logLuminance, numElem, isMin);

  // now we're down to one block left, so reduce it
  // launch one thread for each block in prev step
  shmem_reduce<<<1, numBlock, numBlock * sizeof(float)>>>(d_out, d_intermediate, numElem, isMin);

  float h_out;
  cudaMemcpy(&h_out, d_out, sizeof(float), cudaMemcpyDeviceToHost);
  checkCudaErrors(cudaFree(d_intermediate));
  checkCudaErrors(cudaFree(d_out));
  return h_out;
}

__global__ void shmem_histo(unsigned int* out_histo, const float* d_in, int numBins, int numElem, float minVal, float rangeVal) {
	int myId = threadIdx.x + blockDim.x * blockIdx.x;
  int tid  = threadIdx.x;

  // create a private histogram copy for each thread block
  extern __shared__ unsigned int hist[];
  hist[tid] = 0;  // since BLOCK_SIZE equals numBins, each thread needs to initialize exactly one bin
  __syncthreads();

  // update private histogram
  if (myId < numElem) {
    int bin = (d_in[myId] - minVal) / rangeVal * numBins;
    bin = bin == numBins ? numBins - 1 : bin;  // max value bin is the last of the histo
    atomicAdd(&(hist[bin]), 1);
  }
  __syncthreads();

  atomicAdd(&(out_histo[tid]), hist[tid]);   // since BLOCK_SIZE equals numBins, each thread needs to add exactly one bin to the global bins
}

unsigned int* compute_histogram(const float* const d_logLuminance, int numBins, int numElem, float minVal, float rangeVal) {
  // numBins: 1024, numElem: 393216
	unsigned int* d_histo;
	checkCudaErrors(cudaMalloc(&d_histo, numBins * sizeof(unsigned int)));
	checkCudaErrors(cudaMemset(d_histo, 0, numBins * sizeof(unsigned int)));
	int numBlock = ceil(1.0f * numElem / BLOCK_SIZE);
  int shBytes = numBins * sizeof(unsigned int);
	shmem_histo<<<numBlock, BLOCK_SIZE, shBytes>>>(d_histo, d_logLuminance, numBins, numElem, minVal, rangeVal);
  // cudaDeviceSynchronize();  // block the cpu host until kernel execution finishes.
  // checkCudaErrors(cudaGetLastError());
  return d_histo;
}

// Optimal step efficiency (histogram is a relatively small vector, so Hillis-Steele's is better than Belloch's here)
__global__ void scan_hillis_steele(unsigned int* d_out, const unsigned int* d_in, int numBins) {
  extern __shared__ unsigned int buffer[];
  int tid = threadIdx.x;
  int pout = 0, pin = 1;
  buffer[tid] = tid > 0 ? d_in[tid - 1] : 0; // exclusive scan
  __syncthreads();

  // double buffered
  for (int offset = 1; offset < numBins; offset <<= 1) {
    pout = 1 - pout;
    pin = 1 - pout;
    if (tid >= offset) {
      buffer[numBins * pout + tid] = buffer[numBins * pin + tid] + buffer[numBins * pin + tid - offset];
    } else {
      buffer[numBins * pout + tid] = buffer[numBins * pin + tid];
    }
    __syncthreads();
  }
  d_out[tid] = buffer[pout * numBins + tid];
}

void your_histogram_and_prefixsum(const float* const d_logLuminance,
                                  unsigned int* const d_cdf,
                                  float &min_logLum,
                                  float &max_logLum,
                                  const size_t numRows,
                                  const size_t numCols,
                                  const size_t numBins) {
  /*Here are the steps you need to implement
    1) find the minimum and maximum value in the input logLuminance channel
       store in min_logLum and max_logLum
    2) subtract them to find the range
    3) generate a histogram of all the values in the logLuminance channel using
       the formula: bin = (lum[i] - lumMin) / lumRange * numBins
    4) Perform an exclusive scan (prefix sum) on the histogram to get
       the cumulative distribution of luminance values (this should go in the
       incoming d_cdf pointer which already has been allocated for you)       */

  // 1. Reduce
  int numElem = numRows * numCols;
  min_logLum = reduce(d_logLuminance, numElem, true);
  max_logLum = reduce(d_logLuminance, numElem, false);
  printf("%f %f\n", min_logLum, max_logLum);  // -4.000000 2.404939 0.98ms


  // 2. Range
  float range = max_logLum - min_logLum;

  // 3. histogram
  unsigned int* d_histo = compute_histogram(d_logLuminance, numBins, numElem, min_logLum, range);  // 1.16 ms

  // 4. CDF (scan)
	// Assumption: numBins <= 1024
	scan_hillis_steele<<<1, numBins, 2 * numBins * sizeof(unsigned int)>>>(d_cdf, d_histo, numBins);  // 1.19 ms
}

In [ ]:
# make the cuda project
!make HW3
print("\n====== RESULT OF HW3 =======\n")
!bin/HW3 ../src/HW3/memorial_png_large.gold

In [ ]:
# plot output images
import matplotlib.pyplot as plt
_,ax = plt.subplots(2,2, dpi=150)

ax[0][0].imshow(plt.imread("../src/HW3/memorial_raw_large.png"))
ax[0][0].set_title("original")
ax[0][0].grid(False)

ax[0][1].imshow(plt.imread("HW3_output.png"))
ax[0][1].set_title("output")
ax[0][1].grid(False)

ax[1][0].imshow(plt.imread("HW3_reference.png"))
ax[1][0].set_title("reference")
ax[1][0].grid(False)

ax[1][1].imshow(plt.imread("HW3_differenceImage.png"))
ax[1][1].set_title("difference")
ax[1][1].grid(False)

plt.show()